In [9]:
import os
import gc
from tqdm import tqdm
import illustris_python as il
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import math

# %matplotlib widget
basePath = "./sims.TNG/TNG50-1/output"

Snapshot_Init = 71
Snapshot_End = 99
Lbsg_Indcies_Array = np.array(
    [117251, 300903, 324123, 329508, 382215,
        400973, 419618, 457431, 460722, 490814]
)

h = 0.6774  # km/Mpc

m_dm = 4.5e-5  # E10M_Sum
m_dm_h = m_dm * h

Critial_Density = 1.27e-8  # E10M_Sum/kpc^3
Critial_Density_h = Critial_Density / h**2  # E10M_Sum/(kpc/h)^3


In [2]:
Current_Snapshot = 99
Subhalo_Index = 329508


In [3]:
PartType4_Fields=['Coordinates','Masses','GFM_StellarFormationTime']
PartType4_Dict=il.snapshot.loadSubhalo(basePath,Current_Snapshot,Subhalo_Index,4,PartType4_Fields)


In [ ]:
Subhalo_Dict=il.groupcat.loadSingle


In [5]:
for Coordinates, Masses, StellarFormationTime in zip(PartType4_Dict['Coordinates'], PartType4_Dict['Masses'], PartType4_Dict['GFM_StellarFormationTime']):



[ 7404.55543319 20221.83508105 22046.23399325] 3.4242817e-06 0.33460355


In [10]:
math.pi


3.141592653589793